## Rabi (§TYPE§)
### §PULSE_LENGTH§ns, pi in §PULSE_AMP§V

In [ ]:
f = h5py.File(§HDF5_PATH§, 'r', swmr=True)
pa = f['parameters']
da = f['data']

amp   = da['amp'][()]
data1 = da['I1'][()] + 1j * da['Q1'][()]
data2 = da['I2'][()] + 1j * da['Q2'][()] 
data  = IQ_rot((data1 - data2))
amp2  = np.linspace(amp[0], amp[-1], 1000)

def cosine(t, a_rabi, b, c):
    t_arr = np.array(t)
    return b*np.cos(2*np.pi*t_arr/a_rabi)+c

Nstop = -40 # TODO: compute this hardcoded value (discard non-linear part)

best_popt, best_pcov = opt.curve_fit(cosine, amp[:Nstop], np.real(data[:Nstop]), (amp[-1], 0, 0))
for p in np.linspace(0.5, 20, 40): # 1/2 to 20 periods (step = 1/2)
    popt, pcov = opt.curve_fit(cosine, amp[:Nstop], np.real(data[:Nstop]), (amp[-1]/p, 0, 0))
    if (np.diag(pcov) < np.diag(best_pcov)).all():
        best_popt = popt
        best_pcov = pcov
popt, pcov = best_popt, best_pcov

fig, ax = plt.subplots()

ax.plot(amp,  data.real)
ax.plot(amp2, cosine(amp2, *popt))
ax.legend(('Data', f'Fit: a_rabi = {popt[0]:f}'))

print(f'a_rabi, b, c = {popt}')
print(f'pcov = {np.diag(pcov)}')

result = {'a_rabi': popt[0]}
globals()['result'] = result
result